In [2]:
from datasets import load_dataset, DatasetDict

c:\Users\Felip\OneDrive\Desktop\Dev\finetunian\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
finetunian = load_dataset('audiofolder', data_dir=r'C:\Users\Felip\OneDrive\Desktop\Dev\finetunian\dataset')

In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="english", task="transcribe"
)

In [7]:
finetunian['train'][0]

{'audio': {'path': 'C:\\Users\\Felip\\OneDrive\\Desktop\\Dev\\finetunian\\dataset\\data\\0_speech-accent-stella_1.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.52587891e-05,  1.83105469e-04,  3.96728516e-04]),
  'sampling_rate': 48000},
 'transcription': 'Please call Stella. Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.'}

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate # 16khz
finetunian = finetunian.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [11]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcription"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [14]:
finetunian.column_names['train']

['audio', 'transcription']

In [16]:
finetunian = finetunian.map(
    prepare_dataset, remove_columns=finetunian.column_names["train"], num_proc=1
)

Map: 100%|██████████| 79/79 [00:01<00:00, 54.71 examples/s]


In [17]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [18]:
finetunian["train"] = finetunian["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter: 100%|██████████| 79/79 [00:00<00:00, 2883.31 examples/s]


In [19]:
finetunian["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 79
})

In [23]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [24]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [25]:
import evaluate

metric = evaluate.load("wer")

In [26]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [27]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [28]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="english", task="transcribe", use_cache=True
)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-dv",  # name on the HF Hub
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-6,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=100,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=50,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

c:\Users\Felip\OneDrive\Desktop\Dev\finetunian\venv\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=finetunian["train"],
    eval_dataset=finetunian["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

C:\Users\Felip\AppData\Local\Temp\ipykernel_58404\3984569170.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [37]:
trainer.train()

 25%|██▌       | 25/100 [25:58<1:38:00, 78.40s/it]

{'loss': 5.1349, 'grad_norm': 1471.1719970703125, 'learning_rate': 4e-07, 'epoch': 5.0}


 50%|█████     | 50/100 [59:12<1:05:25, 78.52s/it]

{'loss': 3.8696, 'grad_norm': 363.2127380371094, 'learning_rate': 9e-07, 'epoch': 10.0}


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.proc

...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 1.1981127262115479, 'eval_wer_ortho': 7.105719237435008, 'eval_wer': 4.965753424657534, 'eval_runtime': 265.2667, 'eval_samples_per_second': 0.298, 'eval_steps_per_second': 0.019, 'epoch': 20.0}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
100%|██████████| 100/100 [2:14:42<00:00, 80.82s/it]

{'train_runtime': 8082.3673, 'train_samples_per_second': 0.198, 'train_steps_per_second': 0.012, 'train_loss': 3.1392928314208985, 'epoch': 20.0}


TrainOutput(global_step=100, training_loss=3.1392928314208985, metrics={'train_runtime': 8082.3673, 'train_samples_per_second': 0.198, 'train_steps_per_second': 0.012, 'total_flos': 4.559649325056e+17, 'train_loss': 3.1392928314208985, 'epoch': 20.0})